In [3]:
import sys
sys.path.append('../')
sys.dont_write_bytecode = True
import os
import numpy as np
import scipy
import scipy.ndimage

sys.path.append('/Users/Michael_Saha/Repositories/blahb')
import blahb

from matplotlib.pyplot import *
%matplotlib inline
import numba
sys.path.append('/Users/Michael_Saha/Repositories/')
sys.path.append('./../../../')
from fire_sim import *

import shapefile

In [8]:
shpfile = 'E:/DATA/FotF/burn_perims_new/fire_burn_hist_proj2.shp'
sf = shapefile.Reader(shpfile)

ShapefileException: Unable to open E:/DATA/FotF/burn_perims_new/fire_burn_hist_proj2.shp

In [9]:
shapes = list(sf.shapes())
print(len(shapes))

NameError: name 'sf' is not defined

In [213]:
imshow(blahb.polygon2d(y,x, filled=True).img)

NameError: name 'y' is not defined

In [214]:
fires = []
count = 0
for i in range(len(sf.shapes())):
    year = int(sf.record(i)[0])
    if year < 2000:
        continue
    shp = sf.shape(i)
    x, y = zip(*shp.points)
    x, y = (np.array(x)/10).astype(int), (np.array(y)/10).astype(int)
    
    fires.append(blahb.polygon2d(y, x, filled=True))

NameError: name 'sf' is not defined

In [461]:
def Vector(numba_type):
    if numba_type not in Vector._saved_type:
        
        class _Vector:
            """Dynamically sized arrays in nopython=True mode."""

            def __init__(self):
                self.n = 0
                self.m = 10
                self.arr = np.empty(self.n, dtype=numba_type)

            @property
            def size(self):
                """The number of valid valued."""
                return self.n
            
            def append(self, val):
                if self.n == self.m:
                    self._expand()
                self.arr[self.n] = val
                self.n += 1
            
            def reserve(self, n):
                if n > self.m:
                    temp = np.empty(n, dtype=numba_type)
                    temp[:self.n] = self.arr
                    self.arr = temp
            
            
            def __array__(self):
                return self.arr[:self.n]
            
            def _expand(self):
                self.m *= 1.6
                temp = np.empty(self.m, dtype=numba_type)
                temp[:self.n] = self.arr[:self.n]
                self.arr = temp
            
            def __getitem__(self, idx):
                return self.arr[idx]
        
        spec = [("n",numba.int32), ("m",numba.int32), ("arr",numba_type[:])] # Internals
        Vector._saved_type[numba_type] = numba.jitclass(spec)(_Vector)
    return Vector._saved_type[numba_type]()

Vector._saved_type = dict()




In [465]:
v = Vector(numba.float64)
v.reserve(20)
v.arr

array([ -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57])

In [ ]:
@numba.jit(nopython=True)
def foo(v):
    for i in range(1E6):
        v.append(i)

In [ ]:
%timeit foo(v)

In [460]:
v.append(1)
v.arr

array([  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
         1.00000000e+00,   1.00000000e+00,  -1.36311572e+57,
        -1.36311572e+57,  -1.36311572e+57])